### This script is to correlate TEs to genes based on annotation comparison. It needs to be run for each species seperately and will generate one .csv file per species, containing the gene and associated TE

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
TE_gff_raw  = pd.read_csv('<EDTA_results>/<species>.fasta.mod.EDTA.intact.gff3', sep = '\t', 
                      header = None, index_col=0)
TE_gff = TE_gff_raw[~TE_gff_raw[2].isin(['long_terminal_repeat', 'repeat_region', 'target_site_duplication'])] #delete all rows containing TIR TSD and repeat info

genome_gff = pd.read_csv('<genome_annotation>/<species>.gff3', sep = '\t', engine = 'python', header = None, index_col = 0)

#create path for csv output file. The script will generate a .csv file with each TE associated to a gene and the type of association (upstream, downstream,---)
csv = '<species_output_file>.csv'


## correlate TEs to genes:

In [ ]:
#this version only includes genes as features, leaves UTR annotations etc
genome_gff = genome_gff[genome_gff[2] == 'gene']

In [ ]:
contigs_with_TE = list((set(genome_gff.index)) & (set(TE_gff.index))) #only use contigs where TEs and genes are present, leave the rest
list_of_ignored_contigs = []
subset_genome = genome_gff
subset_genome['location'] = np.nan #to have 'location' in header
subset_TE = TE_gff
subset_genome.iloc[[1]][1:].to_csv(csv, header = True) #make blank csv with header

for c in range(len(contigs_with_TE)): 
    location = []
    df_hit = pd.DataFrame()
    df_hit  = subset_genome.iloc[[0]] #placeholder, delete later
    if contigs_with_TE[c] in TE_gff.index and contigs_with_TE[c] in genome_gff.index:
        subset_TE = TE_gff.loc[[contigs_with_TE[c]]]
        subset_genome = genome_gff.loc[[contigs_with_TE[c]]]
    else:
        list_of_ignored_contigs.append(contigs_with_TE[c]) #for file with list of ignored contigs

    for i in range(len(subset_TE)):
        for m in range(len(subset_genome)):
            if subset_genome.iloc[m][3] < subset_TE.iloc[i][3] < subset_genome.iloc[m][4]: #if TE starts in feature
                df_hit = df_hit.append(subset_genome.iloc[[m]])
                location.append('start')
            if subset_genome.iloc[m][3] < subset_TE.iloc[i][4] < subset_genome.iloc[m][4]: #if TE ends in feature
                df_hit = df_hit.append(subset_genome.iloc[[m]])
                location.append('end')
            if subset_genome.iloc[m][3] < subset_TE.iloc[i][3] < subset_TE.iloc[i][4] < subset_genome.iloc[m][4]: #if TE within feature
                df_hit = df_hit.append(subset_genome.iloc[[m]])
                location.append('within')
            if subset_TE.iloc[i][3] < subset_genome.iloc[m][3] < subset_genome.iloc[m][4] < subset_TE.iloc[i][4] : #if TE over feature
                df_hit = df_hit.append(subset_genome.iloc[[m]])
                location.append('over')  #weird look to keep over when removing duplicates
                
                            
            if subset_genome.iloc[m][3]-3000 < subset_TE.iloc[i][3] < subset_genome.iloc[m][3] : #if TE end up to 3000 bp upstream of gene start
                df_hit = df_hit.append(subset_genome.iloc[[m]])
                if subset_genome.iloc[m][6] == '-': #check for direction of gene
                    location.append('downstream') 
                else:
                    location.append('upstream') 
                
            if subset_genome.iloc[m][3]-3000 < subset_TE.iloc[i][4] < subset_genome.iloc[m][3] : #if TE start up to 3000 bp upstream of gene start
                df_hit = df_hit.append(subset_genome.iloc[[m]])
                if subset_genome.iloc[m][6] == '-': #check for direction of gene
                    location.append('downstream') 
                else:
                    location.append('upstream') 
                
            if subset_genome.iloc[m][4] < subset_TE.iloc[i][3] < subset_genome.iloc[m][4]+3000 : #if TE start up to 3000 bp downstream of gene end
                df_hit = df_hit.append(subset_genome.iloc[[m]])
                if subset_genome.iloc[m][6] == '+': #check for direction of gene
                    location.append('downstream') 
                else:
                    location.append('upstream') 
            if subset_genome.iloc[m][4] < subset_TE.iloc[i][4] < subset_genome.iloc[m][4]+3000 : #if TE end up to 3000 bp downstream of gene end
                df_hit = df_hit.append(subset_genome.iloc[[m]])
                if subset_genome.iloc[m][6] == '+': #check for direction of gene
                    location.append('downstream') 
                else:
                    location.append('upstream') 
                
                
    df_hit_1 = df_hit[1:]
    df_hit_1['location'] = location #add location to df
    df_hit_1.to_csv(csv, mode = 'a', header = False) #open csv and append 
    
#now deduplicate and sort everything
df_hit_1 = pd.read_csv(csv, sep = ',', engine = 'python', header = [0], index_col = 0)
df_hit_1 = df_hit_1[df_hit_1['2'].isin(['gene'])] #keep only ORFs
df_hit_1 = df_hit_1.drop_duplicates(subset='8', keep ='first') 
df_hit_1 = df_hit_1.sort_values(by=['0'], ascending=False) #sort to keep "over"in duplicates
df_hit_1.to_csv(csv, mode = 'w', header = True) #open csv and append 

### now add Orthogroup of analyzed species

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv(csv)

df ['8'] = [i.split('.')[0][3:] for i in df['8']] #reformat ID compatible with Orthofinder Results

ogs_list = []
orthogroups = pd.read_csv('<EDTA_results></Orthogroups/Orthogroups.tsv', sep = '\t', dtype={'user_id': int})
for i in df['8']: #compare our csv file with tsv file from OrthoFinder
    try:
        A = (orthogroups['<species>'])
        match = A[A.str.contains(i, na = False)] #does the csv column contain an OG from Orthofinder?
        ogs_list.append(orthogroups.loc[int(str(match).split(' ')[0])]['Orthogroup']) 
    except ValueError:
        ogs_list.append('no_OG')
        continue
df ['orthogroups'] = ogs_list
df.to_csv(csv, mode = 'w', header = True) #open csv and append 

### extract CDS files from found genes associated to TEs

In [ ]:
#now get seqs from the CDS found:
#open CDS file:

#make cds dict
cds = cds_raw.split('>')
cds_dic = {}
for i in range(len(cds))[1:]:
    cds_dic[cds[i].split('\n')[0].split(' ')[0]] = ''.join(cds[i].split('\n')[1:]).upper()
    
#get ID's from genes spanned by TE
ids_spanned = [i.split('=')[1] for i in df['ID']] 
#now write CDS of these genes to fasta file
write = []
for i in ids_spanned:
    write.append('>')
    write.append(i)
    write.append('\n')
    write.append(cds_dic[i])
    write.append('\n')
write = ''.join(write)
text_file = open('<output_cds>.fasta', 'w')
text_file.write(write)
text_file.close()

### the above file can now be uploaded to Mercator 